# 05i - Vertex AI > Training > Hyperparameter Tuning Jobs - With Custom Container

### 05 Series Overview
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the `05` notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this `05a-05i` series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python script (`05a`), python source distribution (`05b`), and custom container (`05c`)
-  Training Pipeline that trains and saves models from a python script (`05d`), python source distribution (`05e`), and custom container (`05f`)
-  Hyperparameter Tuning Jobs from a python script (`05g`), python source distribution (`05h`), and custom container (`05i`)

### This Notebook (`05i`): An extension of `05c` with Hyperparmeter Tuning - And Tensorboard HParams  
This notebook trains the same Tensorflow Keras model from 05 by first modifying and saving the training code to a python script.  Then a Python source distribution is built containing the script.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this job gives an example of making the shift.  

The source distribution is then used as an input for a Vertex AI Training Custom Job that is also assigned compute resources and a container (pre-built) for executing the training in a managed service.  

The Custom Job is then used as the input for a Vertex AI Training Hyperparameter Tuning Job.  This runs and manages the tuning loops for the number of trials in each loop, collects the metric(s) and manages the parameters with the search algorithm for parameter modification. 

The training can be reviewed with Vertex AI's managed Tensorboard under Experiments > Experiments, or by clicking on the `05i...` job under Training > Hyperparameter Tuning Jobs and then clicking the 'Open Tensorboard' link.  **Click on the HParams tab in Tensorboard to review the hyperparameters and metrics.**

<img src="architectures/overview/Training.png">

---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05i_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05i_console.png">